In [112]:
from pyspark.sql import SparkSession
from pyspark.sql import Row
import  pyspark.sql.functions as fun 
import numpy as np
from pyspark.sql.types import *

In [113]:
spark = SparkSession.builder.getOrCreate()

In [114]:
sc = spark.sparkContext

Create an RDD from a list of numbers (1,50) using numpy methods

In [146]:
rdd1 = sc.parallelize(np.arange(1,50))
rdd1.collect()

[1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49]

Find the sum, average, maximum, minimum, and count

In [116]:
#Sum
rdd1.sum()

1225

In [117]:
#avg
avg = rdd1.sum() / rdd1.count()
avg

25.0

In [118]:
#Count
rdd1.count()

49

In [119]:
#Min
rdd1.min()

1

In [120]:
#Max
rdd1.max()

49

Count how many numbers are even vs. odd.

In [121]:
even = rdd1.filter(lambda x: x % 2==0).count()
odd = rdd1.filter(lambda x: x % 2!=0).count()
print(f"ODD Numbers: {odd} Even Numbers: {even}")

ODD Numbers: 25 Even Numbers: 24


You have the following data of people info ('Name', 'Age'), answer the following questions

In [122]:
people_data = [("Nada ", 25), ("Mona", 30), ("Ahmed", 35), ("Khaled", 40),("Ahmed", 35), ('Nada ', 25)]
rdd_people = sc.parallelize(people_data)
rdd_people.collect()

[('Nada ', 25),
 ('Mona', 30),
 ('Ahmed', 35),
 ('Khaled', 40),
 ('Ahmed', 35),
 ('Nada ', 25)]

Find the oldest person

In [123]:
oldest = rdd_people.max(lambda x:x[1])
oldest

('Khaled', 40)

Compute the average age

In [124]:
average = rdd_people.map(lambda x: x[1]).mean()
average

31.666666666666668

Group all the names by their age

In [125]:
rdd_people.groupBy(lambda x:x[1]).mapValues(list).collect()

[(40, [('Khaled', 40)]),
 (25, [('Nada ', 25), ('Nada ', 25)]),
 (35, [('Ahmed', 35), ('Ahmed', 35)]),
 (30, [('Mona', 30)])]

Take the following text and put it in a text file named russia.txt and load it into rdd

"Russia is the largest country in the world by land area
Moscow is the capital city of Russia
The Russian language is one of the most widely spoken languages in the world
Russia is known for its rich history and culture
The Trans-Siberian Railway is the longest railway line in the world
Russia has a strong tradition in literature, music and ballet
The country is famous for its cold winters and vast landscapes
Russia is a major player in global energy production
"

In [126]:
text = """Russia is the largest country in the world by land area
Moscow is the capital city of Russia
The Russian language is one of the most widely spoken languages in the world
Russia is known for its rich history and culture
The Trans-Siberian Railway is the longest railway line in the world
Russia has a strong tradition in literature, music and ballet
The country is famous for its cold winters and vast landscapes
Russia is a major player in global energy production
"""

with open("russia.txt", "w") as f:
    f.write(text)


In [127]:
rdd_russia = sc.textFile("russia.txt")
rdd_russia.collect()


['Russia is the largest country in the world by land area',
 'Moscow is the capital city of Russia',
 'The Russian language is one of the most widely spoken languages in the world',
 'Russia is known for its rich history and culture',
 'The Trans-Siberian Railway is the longest railway line in the world',
 'Russia has a strong tradition in literature, music and ballet',
 'The country is famous for its cold winters and vast landscapes',
 'Russia is a major player in global energy production']

Count the total number of lines.

In [128]:
rdd_russia.count()

8

Count how many lines contain the word "Russia"

In [129]:
rdd_russia.filter(lambda line:"Russia" in line).count()

6

Find the most 5 frequent word in the file.

In [130]:
top5   = rdd_russia.flatMap(lambda line: line.split()) \
                                    .map(lambda w: (w, 1)) \
                                    .reduceByKey(lambda a,b: a+b) \
                                    .takeOrdered(5,lambda x: -x[1]) 
print(top5)

[('is', 7), ('the', 7), ('Russia', 5), ('in', 5), ('world', 3)]


Tokenize words

In [131]:
Tokenize_words = rdd_russia.flatMap(lambda line: line.split()).collect()
print(Tokenize_words)

['Russia', 'is', 'the', 'largest', 'country', 'in', 'the', 'world', 'by', 'land', 'area', 'Moscow', 'is', 'the', 'capital', 'city', 'of', 'Russia', 'The', 'Russian', 'language', 'is', 'one', 'of', 'the', 'most', 'widely', 'spoken', 'languages', 'in', 'the', 'world', 'Russia', 'is', 'known', 'for', 'its', 'rich', 'history', 'and', 'culture', 'The', 'Trans-Siberian', 'Railway', 'is', 'the', 'longest', 'railway', 'line', 'in', 'the', 'world', 'Russia', 'has', 'a', 'strong', 'tradition', 'in', 'literature,', 'music', 'and', 'ballet', 'The', 'country', 'is', 'famous', 'for', 'its', 'cold', 'winters', 'and', 'vast', 'landscapes', 'Russia', 'is', 'a', 'major', 'player', 'in', 'global', 'energy', 'production']


Remove stopwords (a, the, is, to, in, of). 

In [132]:
stopwords = {"a", "the", "is", "to", "in", "of"}

Remove_stopwords = rdd_russia.flatMap(lambda line: line.split())\
                                      .map(lambda w: w.lower()) \
                                      .filter(lambda w: w not in stopwords).collect()

print(Remove_stopwords)

['russia', 'largest', 'country', 'world', 'by', 'land', 'area', 'moscow', 'capital', 'city', 'russia', 'russian', 'language', 'one', 'most', 'widely', 'spoken', 'languages', 'world', 'russia', 'known', 'for', 'its', 'rich', 'history', 'and', 'culture', 'trans-siberian', 'railway', 'longest', 'railway', 'line', 'world', 'russia', 'has', 'strong', 'tradition', 'literature,', 'music', 'and', 'ballet', 'country', 'famous', 'for', 'its', 'cold', 'winters', 'and', 'vast', 'landscapes', 'russia', 'major', 'player', 'global', 'energy', 'production']


Count the frequency of each word

In [133]:
word_counts  = rdd_russia.flatMap(lambda line: line.split()) \
                                    .map(lambda w: (w, 1)) \
                                    .reduceByKey(lambda a,b: a+b).collect()

print(word_counts)

[('Russia', 5), ('largest', 1), ('country', 2), ('world', 3), ('by', 1), ('land', 1), ('area', 1), ('capital', 1), ('of', 2), ('language', 1), ('most', 1), ('widely', 1), ('known', 1), ('for', 2), ('history', 1), ('and', 3), ('Trans-Siberian', 1), ('Railway', 1), ('line', 1), ('literature,', 1), ('music', 1), ('famous', 1), ('cold', 1), ('winters', 1), ('landscapes', 1), ('player', 1), ('energy', 1), ('production', 1), ('is', 7), ('the', 7), ('in', 5), ('Moscow', 1), ('city', 1), ('The', 3), ('Russian', 1), ('one', 1), ('spoken', 1), ('languages', 1), ('its', 2), ('rich', 1), ('culture', 1), ('longest', 1), ('railway', 1), ('has', 1), ('a', 2), ('strong', 1), ('tradition', 1), ('ballet', 1), ('vast', 1), ('major', 1), ('global', 1)]


In [134]:
schema = 'id integer, name string, age integer, salary integer' 
data = [
    (1, "Ali", 25, 4000),
    (2, "Mariam", 30, 6000),
    (3, "Omar", 35, 7000),
    (4, "Sara", 28, 5000),
    (5, "Omar", 25, 6500),
    (6, "Mariam", 26, 7500)
]

df = spark.createDataFrame(data,schema)

Show schema and first 2 rows

In [135]:
df.printSchema()

root
 |-- id: integer (nullable = true)
 |-- name: string (nullable = true)
 |-- age: integer (nullable = true)
 |-- salary: integer (nullable = true)



In [136]:
df.show(2)

+---+------+---+------+
| id|  name|age|salary|
+---+------+---+------+
|  1|   Ali| 25|  4000|
|  2|Mariam| 30|  6000|
+---+------+---+------+
only showing top 2 rows



Select only name and salary

In [137]:
df.select("name", "salary").show()

+------+------+
|  name|salary|
+------+------+
|   Ali|  4000|
|Mariam|  6000|
|  Omar|  7000|
|  Sara|  5000|
|  Omar|  6500|
|Mariam|  7500|
+------+------+



Find the average salary

In [138]:
df.select(fun.avg("salary")).show()

+-----------+
|avg(salary)|
+-----------+
|     6000.0|
+-----------+



Filter employees older than 28

In [139]:
df.filter(df.age >28).show()

+---+------+---+------+
| id|  name|age|salary|
+---+------+---+------+
|  2|Mariam| 30|  6000|
|  3|  Omar| 35|  7000|
+---+------+---+------+



Count distinct values in the name column

In [140]:
df.select("name").distinct().count()

4

Group by a the name column and find average salary

In [141]:
df.groupBy("name").agg(fun.avg("salary")).show()

+------+-----------+
|  name|avg(salary)|
+------+-----------+
|   Ali|     4000.0|
|Mariam|     6750.0|
|  Omar|     6750.0|
|  Sara|     5000.0|
+------+-----------+



In [142]:
df1 = spark.read.csv("/data/NullData.csv", header=True, inferSchema=True) #this file in shared folder
df1.show()

+----+-----+-----+
|  Id| Name|Sales|
+----+-----+-----+
|emp1| John| NULL|
|emp2| NULL| NULL|
|emp3| NULL|  345|
|emp4|Cindy|  456|
+----+-----+-----+



Find the avg sales 

In [143]:
avg_sales = df1.select(fun.avg("sales").alias("Average Sales")).first()[0]
print("Average sales:", avg_sales)

Average sales: 400.5


Replace null name with 'Unknown' and sales with the avg sales of the column 

In [144]:
df1_filled = df1.fillna({"name": "Unknown", "sales": avg_sales})
df1_filled.show()


+----+-------+-----+
|  Id|   Name|Sales|
+----+-------+-----+
|emp1|   John|  400|
|emp2|Unknown|  400|
|emp3|Unknown|  345|
|emp4|  Cindy|  456|
+----+-------+-----+

